In [582]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
np.random.seed(1)

In [583]:
train = pd.read_csv('cs-training.csv')
test = pd.read_csv('cs-test.csv')
train.fillna(0,inplace=True)
test.fillna(0,inplace=True)

In [584]:
train.columns = ['Id','SeriousDlqin2yrs','RevolvingUtilizationOfUnsecuredLines','age','NumberofTimes30to59'
                ,'DebtRatio','MonthlyIncome','NumberofOpenCreditLinesandLoans','NumberofTimes90'
                ,'NumberRealEstateLoansOrLines','NumberofTimes60to89','NumberOfDependents']
test.columns = ['Id','SeriousDlqin2yrs','RevolvingUtilizationOfUnsecuredLines','age','NumberofTimes30to59'
                ,'DebtRatio','MonthlyIncome','NumberofOpenCreditLinesandLoans','NumberofTimes90'
                ,'NumberRealEstateLoansOrLines','NumberofTimes60to89','NumberOfDependents']

In [585]:
test_id = test['Id']
target = train['SeriousDlqin2yrs']
cols = ['RevolvingUtilizationOfUnsecuredLines','age','NumberofTimes30to59'
                ,'DebtRatio','MonthlyIncome','NumberofOpenCreditLinesandLoans','NumberofTimes90'
                ,'NumberRealEstateLoansOrLines','NumberofTimes60to89','NumberOfDependents']
train = train[cols]
test = test[cols]
train.head()

,RevolvingUtilizationOfUnsecuredLines,age,NumberofTimes30to59,DebtRatio,MonthlyIncome,NumberofOpenCreditLinesandLoans,NumberofTimes90,NumberRealEstateLoansOrLines,NumberofTimes60to89,NumberOfDependents
0,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [586]:
x_train,x_test,y_train,y_test = train_test_split(train,target,test_size=0.3,shuffle=True,random_state=26,stratify=target)
y_train = y_train.values.reshape((1,-1))
y_test = y_test.values.reshape((1,-1))
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(105000, 10)
(45000, 10)
(1, 105000)
(1, 45000)


In [587]:
scaler = StandardScaler()
train_std = scaler.fit_transform(x_train)
test_std = scaler.fit_transform(x_test)
train_std = train_std.T
test_std = test_std.T
print(train_std.shape)
test_std.shape

(10, 105000)


(10, 45000)

In [588]:
def initialize_parameters_deep(nn_dims):
    
    np.random.seed(1)
    parameters={}
    
    L = len(nn_dims)
    for l in range(1,L):
        parameters['W'+str(l)] = np.random.randn(nn_dims[l],nn_dims[l-1]) / np.sqrt(nn_dims[l-1])
        parameters['b'+str(l)] = np.zeros((nn_dims[l],1))
        
    return parameters

In [589]:
def linear_forward(A,W,b):
    
   # print(W.shape)
   # print(A.shape)
    Z = np.dot(W,A) + b
    cache = (A,W,b)
    
    return Z,cache

In [590]:
def relu(Z):
    A = np.maximum(0,Z)
    cache = Z
    return A,cache

In [591]:
def sigmoid(Z):
    A = 1/(1+np.exp(-Z))
    cache = Z
    return A,cache

In [592]:
def linear_activation_forward(A_prev,W,b,activation):
    
    if activation == 'relu':
        Z,linear_cache = linear_forward(A_prev,W,b)
        A,activation_cache = relu(Z)
        
    elif activation == 'sigmoid':
        Z,linear_cache = linear_forward(A_prev,W,b)
        A,activation_cache = sigmoid(Z)
        
    cache = (linear_cache,activation_cache)
    
    return A,cache
        

In [593]:
def compute_cost(AL,Y):
    
    m =  Y.shape[1]
    
    cost = -1 / m * (np.dot(Y,np.log(AL).T) + np.dot(1-  Y ,np.log(1-AL).T))
    
    cost = np.squeeze(cost)
    
    return cost

In [594]:
def L_model_forward(X,parameters):
    
    caches=[]
    A = X
    L = len(parameters) // 2
    
    for l in range(1,L):
        A_prev = A
        A,cache = linear_activation_forward(A_prev,parameters['W'+ str(l)],parameters['b' + str(l)],'relu')
        caches.append(cache)
        
    AL,cache = linear_activation_forward(A,parameters['W'+ str(L)],parameters['b' + str(L)],'sigmoid')
    caches.append(cache)
    
    return AL,caches
    
    

In [595]:
def relu_backward(dA,cache):
    Z = cache
    dZ = np.array(dA,copy = True)
    dZ[Z <= 0] = 0
    
    return dZ

In [596]:
def sigmoid_backward(dA,cache):
    
    Z = cache
    
    s = 1 / (1 + np.exp(-Z))
    dZ = dA * s * (1 -s)
    return dZ

In [597]:
def linear_backward(dZ,cache):
    
    A_prev,W,b = cache
    m = A_prev.shape[1]
    
    dW = np.dot(dZ,A_prev.T) / m
    db = np.sum(dZ,axis=1,keepdims=True) / m
    dA_prev = np.dot(W.T,dZ)
    
    return dA_prev,dW,db
    

In [598]:
def linear_activation_backward(dA,cache,activation):
    
    linear_cache,activation_cache = cache
    
    if activation =='relu':
        dZ = relu_backward(dA,activation_cache)
        dA_prev,dW,db = linear_backward(dZ,linear_cache)
    elif activation =='sigmoid':
        dZ = sigmoid_backward(dA,activation_cache)
        dA_prev,dW,db = linear_backward(dZ,linear_cache)
        
    return dA_prev,dW,db

In [599]:
def L_model_backward(AL,Y,caches):
    
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    
    dAL = - (np.divide(Y,AL) - np.divide(1 - Y,1 - AL))
    
    current_cache = caches[L - 1]
    dA_prev_temp,dW_temp,db_temp = linear_activation_backward(dAL,current_cache,'sigmoid')
    grads["dA" + str(L-1)] = dA_prev_temp
    grads["dW" + str(L)] = dW_temp
    grads["db" + str(L)] = db_temp
    # YOUR CODE ENDS HERE
    
    # Loop from l=L-2 to l=0
    for l in reversed(range(L-1)):
        # lth layer: (RELU -> LINEAR) gradients.
        # Inputs: "grads["dA" + str(l + 1)], current_cache". Outputs: "grads["dA" + str(l)] , grads["dW" + str(l + 1)] , grads["db" + str(l + 1)] 
        #(approx. 5 lines)
        # current_cache = ...
        # dA_prev_temp, dW_temp, db_temp = ...
        # grads["dA" + str(l)] = ...
        # grads["dW" + str(l + 1)] = ...
        # grads["db" + str(l + 1)] = ...
        # YOUR CODE STARTS HERE
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 1)],current_cache,'relu')
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
        # YOUR CODE ENDS HERE

    return grads

In [600]:
def update_parameters(params,grads,learning_rate):
    
    parameters = params.copy()
    L = len(parameters) // 2 # number of layers in the neural network
    #print("Learning Rate",learning_rate)

    # Update rule for each parameter. Use a for loop.
    #(≈ 2 lines of code)
    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]
        
    return parameters

In [601]:
def L_layer_model(X,y,layers_dims,learning_rate=0.0075,num_iterations=3000,print_cost=False):
    
    np.random.seed(1)
    costs=[]
    
    parameters = initialize_parameters_deep(layers_dims)
    
    for i in range(0,num_iterations):
        
        AL,caches = L_model_forward(X,parameters)
        
        cost = compute_cost(AL,y)
        
        grads = L_model_backward(AL,y,caches)
        
        parameters = update_parameters(parameters,grads,learning_rate)
        
        # Print the cost every 100 iterations
        if print_cost and i % 100 == 0 or i == num_iterations - 1:
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if i % 100 == 0 or i == num_iterations:
            costs.append(cost)
    
    return parameters,costs
        
        

In [602]:
layers_dims = [10, 20, 7, 5, 1] #  4-layer model
#layers_dims = [10, 20, 10,1] #  4-layer model
#L_layer_model(train_x,train_y,layers_dims,num_iterations=1,print_cost=True)
parameters, costs = L_layer_model(train_std, y_train, layers_dims,learning_rate=0.1, num_iterations = 2500, print_cost = True)

Cost after iteration 0: 0.7593582356562509
Cost after iteration 100: 0.24847616243315934
Cost after iteration 200: 0.24202705132885233
Cost after iteration 300: 0.2377842783504525
Cost after iteration 400: 0.23457587596090676
Cost after iteration 500: 0.23214617690096678
Cost after iteration 600: 0.22969653705418072
Cost after iteration 700: 0.22706308526311697
Cost after iteration 800: 0.22433527297183528
Cost after iteration 900: 0.2215146917114944
Cost after iteration 1000: 0.21840651986842122
Cost after iteration 1100: 0.21508644249096645
Cost after iteration 1200: 0.21149713704945522
Cost after iteration 1300: 0.2082708627589642
Cost after iteration 1400: 0.20532830109854894
Cost after iteration 1500: 0.20269677793228338
Cost after iteration 1600: 0.20058928015739794
Cost after iteration 1700: 0.19873004525772361
Cost after iteration 1800: 0.19708125526810777
Cost after iteration 1900: 0.1956092390201407
Cost after iteration 2000: 0.1943941923984134
Cost after iteration 2100: 0.19

In [603]:
def predict(X, y, parameters):
    
    m = X.shape[1]
    n = len(parameters) // 2 # number of layers in the neural network
    p = np.zeros((1,m))
    
    # Forward propagation
    probas, caches = L_model_forward(X, parameters)

    
        
    return probas


In [604]:
pred = np.array(predict(test_std,y_test,parameters)[0])
actual = np.array(y_test[0])
pred

array([0.17427459, 0.10360591, 0.01471627, ..., 0.0341742 , 0.01939018,
       0.02649839])

In [605]:
max(pred)

0.9418625271697675

In [606]:
roc_auc_score(actual,pred)

0.8082726402040105